## Seasonal Decomposition of Time Series (STL) with the ESD (Extreme Studentized Deviate)

This method combines time series decomposition with a statistical test for detecting anomalies. It works well for data with seasonality and trend components.


In [3]:
import pandas as pd
import numpy as np
# Generate random revenue data for 3 clients over 45 months
num_clients = 3
num_months = 45
months = pd.date_range(start='2018-01', periods=num_months, freq='M')
client_data = {f"client_{i+1}": pd.DataFrame({'Month': months, 'Revenue': np.random.uniform(800 + i * 500, 1500 + i * 500, num_months)}) for i in range(num_clients)}


In [2]:
import statsmodels.api as sm
from scipy.stats import t

In [4]:
def analyze_anomalies_stl_esd(client_name, client_df, seasonal_period=12, alpha=0.05):
    # Decompose the time series
    revenue_series = pd.Series(client_df['Revenue'].values, index=client_df['Month'])
    stl = sm.tsa.STL(revenue_series, period=seasonal_period)
    decomposed = stl.fit()

    # Calculate the residuals
    client_df['Residuals'] = decomposed.resid

    # ESD test for anomalies
    anomalies = []
    residuals = client_df['Residuals']
    for _ in range(int(alpha * len(residuals))):
        max_deviation = np.abs(residuals).max()
        max_index = residuals.index[np.abs(residuals).argmax()]
        test_stat = max_deviation / residuals.std()

        p = 1 - t.cdf(test_stat, df=len(residuals) - 2)
        critical_value = t.ppf(1 - alpha / (2 * len(residuals)), df=len(residuals) - 2)

        if max_deviation > critical_value:
            anomalies.append((max_index, residuals[max_index]))
            residuals = residuals.drop(max_index)
        else:
            break

    # Print anomalies
    print(f"Anomalies for {client_name}:")
    for index, anomaly_value in anomalies:
        print(f"Month {client_df['Month'][index]}: Residual = {anomaly_value:.2f}")
    print("\n")

In [5]:
for client_name, client_df in client_data.items():
    analyze_anomalies_stl_esd(client_name, client_df)

Anomalies for client_1:


Anomalies for client_2:


Anomalies for client_3:




- The `analyze_anomalies_stl_esd` function first decomposes the time series into its trend, seasonal, and residual components using the `STL` class from the `statsmodels.tsa` module.
- Then, it applies the ESD test to the residual component of the decomposed time series to detect anomalies.
- The function prints the anomalous data points.

Keep in mind that you should replace the generated data with your actual dataset. The `seasonal_period` and `alpha` parameters in the `analyze_anomalies_stl_esd` function control the seasonal period (default is 12 months) and the significance level for the ESD test (default is 0.05). You can adjust these parameters based on your specific use case.

The STL with ESD method is more robust to seasonality and trend components and can capture complex patterns compared to the Z-score and IQR methods. However, it assumes that the time
